In [45]:
def print_matrix(input_file, A):
    """Print the matrix with the (0,0) entry in the top left
    corner. Will label the rows by the sequence and add in the
    0-row if appropriate."""
    with open(input_file,'r') as f:
        sequences = f.readlines()
        x = sequences[0].strip()
        y = sequences[1].strip()
    f.close()
    # decide whether there is a 0th row/column
    if len(x) == len(A):
        print ("%5s" % (" "),end = '')
    else:
        print ("%5s %5s" % (" ","*"),end = '')
        y = "*" + y

    # print the top row
    for c in x:
        print ("%5s" % (c),end = '')
    print()

    for j in range(len(A[0])):
        print ("%5s" % (y[j]),end = '')
        for i in range(len(A)):
            print ("%5.0f" % (A[i][j]),end = '')
        print()


def global_align(input_file, gap_opening_penalty, gap_extend_penalty,match_score,mismatch_score):
    """Global alignment with affine penalties. We assume we are maximizing."""
    # TO-DO 1: Read the sequences from the given input file
    with open(input_file,'r') as f:
      seq = f.readlines()
      x = seq[0].strip()
      y = seq[1].strip()
      x_size = len(x) + 1 # add 1 because first coordinate is generated by gaps in both x_size and y_size of the matrix
      y_size = len(y) + 1
    f.close()
    # TO-DO 2: Create the matrix
    scoring_matrix = [] # x_size = row, y_size = column
    for i in range(x_size):
      a = []
      for j in range(y_size):
        a.append(0)
      scoring_matrix.append(a)
    # Create the paths we visited as a matrix to reach the alignment
    path_matrix = [] 
    for i in range(x_size):
      pathList = []
      for j in range(y_size):
        pathList.append('?') # firstly our path matrix is full of gaps
      path_matrix.append(pathList)
    # TO-DO 3: Fill up the score matrix
    scoring_matrix[1][0] += gap_opening_penalty 
    for i in range(x_size - 2):
      scoring_matrix[i+2][0] =  scoring_matrix[i+1][0] + gap_extend_penalty
    scoring_matrix[0][1] += gap_opening_penalty 
    for j in range(y_size - 2):
      scoring_matrix[0][j+2] =  scoring_matrix[0][j+1] + gap_extend_penalty
    for i in range(x_size):
        for j in range(y_size):
          if (i > 0 and j > 0):
            up = scoring_matrix[i][j-1] + gap_extend_penalty
            left = scoring_matrix[i-1][j] + gap_extend_penalty
            if (seq[0][i-1] == seq[1][j-1]):
              score = match_score
            else:
              score = mismatch_score
            diag = scoring_matrix[i-1][j-1] + score
            if (up >= left and up >= diag):
              path_matrix[i][j] = 'u'
            elif (left >= up and left >= diag):
              if (left > up):
                path_matrix[i][j] = 'l'
              if (path_matrix[i][j] == 'u'):
                if (scoring_matrix[i-1][j] > up):
                  path_matrix[i][j] = 'l'
            elif (diag >= left and diag >= up):
              if (diag > left and diag > up):
                path_matrix[i][j] = 'd'
              if (path_matrix[i][j] == 'u'):
                if (scoring_matrix[i-1][j-1] > scoring_matrix[i-1][j]):
                  path_matrix[i][j] = 'd'
              if (path_matrix[i][j] == 'l'):
                if (scoring_matrix[i-1][j-1] > scoring_matrix[i][j-1]):
                  path_matrix[i][j] = 'd'
            scoring_matrix[i][j] = max(up,left,diag)


    # TO-DO 4: Find the maximum score and its indexes 
    values = []
    max_score = scoring_matrix[0][0]
    for i in range(x_size):
      for j in range(y_size):
        if (i > 0 and j > 0):
          if (scoring_matrix[i][j] > max_score):
             max_score = scoring_matrix[i][j]
             max_xcoor = i
             max_ycoor = j
    values.append(max_score)
    aln1 = ""
    aln2 = ""         
    while (max_score > 0):
      if (path_matrix[max_xcoor][max_ycoor] == '?'):
        break
      if (path_matrix[max_xcoor][max_ycoor] == 'u'):
        values.append(scoring_matrix[max_xcoor][max_ycoor-1])
        
        aln1 += "_"
        aln2 += seq[1][max_ycoor-1]
        max_ycoor -= 1
      elif (path_matrix[max_xcoor][max_ycoor] == 'l'):
        values.append(scoring_matrix[max_xcoor-1][max_ycoor])
        aln2 += "_"
        aln1 = seq[0][max_xcoor-1]
        
        max_xcoor -= 1
      elif (path_matrix[max_xcoor][max_ycoor] == 'd'):
        values.append(scoring_matrix[max_xcoor-1][max_ycoor-1])
        
        aln1 += seq[0][max_xcoor-1]
        aln2 += seq[1][max_ycoor-1]
        max_ycoor -= 1
        max_xcoor -= 1  
      max_score = scoring_matrix[max_xcoor][max_ycoor]   
    del values[-1]
    aln1 = aln1[::-1]
    aln2 = aln2[::-1] 
    
    # TO-DO 5: Write your scoring matrix, match score, mismatch score, gap scores, and alignment to an output file
    gf = open("global_aln_output.txt.", "w")
    for line in scoring_matrix:
      gf.writelines([str(line), "\n"])
    gf.writelines(["Gap Extension Score: ", str(gap_extend_penalty), "\n"])
    gf.writelines(["Gap Opening Score: ", str(gap_opening_penalty), "\n"])

    gf.writelines(["Match Score: ", str(match_score), "\n"])
    gf.writelines(["Mismatch Score: ", str(mismatch_score), "\n"])
    gf.writelines(["Alignment 1: ", aln1, "\n"])
    gf.writelines(["Alignment 2: ", aln2])
    gf.close()
    # Use the print_matrix function given above to print your filled matrix
    print_matrix(input_file,scoring_matrix) 

#run the global alignment function

# Differences in affine and linear
global_align("global_aln_test4.txt", gap_opening_penalty=-15, gap_extend_penalty=-7, match_score=10, mismatch_score=-2) # affine
global_align("global_aln_test4.txt", gap_opening_penalty=-15, gap_extend_penalty=-15, match_score=10, mismatch_score=-2) # linear
global_align("global_aln_test.txt", gap_opening_penalty=-15, gap_extend_penalty=-7, match_score=10, mismatch_score=-2)





          *    A    A    G    A    C    G
    *    0  -15  -22  -29  -36  -43  -50
    C  -15   -2   -9  -16  -23  -26  -33
    A  -22   -5    8    1   -6  -13  -20
    G  -29  -12    1   18   11    4   -3
    A  -36  -19   -2   11   28   21   14
    G  -43  -26   -9    8   21   26   31
    C  -50  -33  -16    1   14   31   24
    T  -57  -40  -23   -6    7   24   29
    C  -64  -47  -30  -13    0   17   22
          *    A    A    G    A    C    G
    *    0  -15  -30  -45  -60  -75  -90
    C  -15   -2  -17  -32  -47  -50  -65
    A  -30   -5    8   -7  -22  -37  -52
    G  -45  -20   -7   18    3  -12  -27
    A  -60  -35  -10    3   28   13   -2
    G  -75  -50  -25    0   13   26   23
    C  -90  -65  -40  -15   -2   23   24
    T -105  -80  -55  -30  -17    8   21
    C -120  -95  -70  -45  -32   -7    6
          *    C    G    T    G    A    A    T    T    C    A    T
    *    0  -15  -22  -29  -36  -43  -50  -57  -64  -71  -78  -85
    G  -15   -2   -5  -12  -19  -26  -33  -40

2.3

c) In linear penalty, each gap recieves the same score of penalty but in affine penalty there is an opening score for the adjacents of the origin and then it keeps growing as the extension score.

In [46]:
def print_matrix(input_file, A):
    """Print the matrix with the (0,0) entry in the top left
    corner. Will label the rows by the sequence and add in the
    0-row if appropriate."""
    with open(input_file,'r') as f:
        sequences = f.readlines()
        x = sequences[0].strip()
        y = sequences[1].strip()
    f.close()
    # decide whether there is a 0th row/column
    if len(x) == len(A):
        print ("%5s" % (" "),end = '')
    else:
        print ("%5s %5s" % (" ","*"),end = '')
        y = "*" + y

    # print the top row
    for c in x:
        print ("%5s" % (c),end = '')
    print()

    for j in range(len(A[0])):
        print ("%5s" % (y[j]),end = '')
        for i in range(len(A)):
            print ("%5.0f" % (A[i][j]),end = '')
        print()


def local_align(input_file, gap_opening_penalty, gap_extend_penalty,match_score,mismatch_score):
    """Global alignment with affine penalties. We assume we are maximizing."""
    # TO-DO 1: Read the sequences from the given input file
    with open(input_file,'r') as f:
      seq = f.readlines()
      x = seq[0].strip()
      y = seq[1].strip()
      x_size = len(x) + 1 # add 1 because first coordinate is generated by gaps in both x_size and y_size of the matrix
      y_size = len(y) + 1
    f.close()
    # TO-DO 2: Create the matrix
    scoring_matrix = [] # x_size = row, y_size = column
    for i in range(x_size):
      a = []
      for j in range(y_size):
        a.append(0)
      scoring_matrix.append(a)
    # Create the paths we visited as a matrix to reach the alignment
    path_matrix = [] 
    for i in range(x_size):
      pathList = []
      for j in range(y_size):
        pathList.append('?') # firstly our path matrix is full of gaps
      path_matrix.append(pathList)
    # TO-DO 3: Fill up the score matrix
    scoring_matrix[1][0] += gap_opening_penalty 
    for i in range(x_size - 2):
      scoring_matrix[i+2][0] =  scoring_matrix[i+1][0] + gap_extend_penalty
    scoring_matrix[0][1] += gap_opening_penalty 
    for j in range(y_size - 2):
      scoring_matrix[0][j+2] =  scoring_matrix[0][j+1] + gap_extend_penalty
    for i in range(x_size):
        for j in range(y_size):
          if (i > 0 and j > 0):
            up = scoring_matrix[i][j-1] + gap_extend_penalty
            left = scoring_matrix[i-1][j] + gap_extend_penalty
            if (seq[0][i-1] == seq[1][j-1]):
              score = match_score
            else:
              score = mismatch_score
            diag = scoring_matrix[i-1][j-1] + score
            if (up >= left and up >= diag):
              path_matrix[i][j] = 'u'
            elif (left >= up and left >= diag):
              if (left > up):
                path_matrix[i][j] = 'l'
              if (path_matrix[i][j] == 'u'):
                if (scoring_matrix[i-1][j] > up):
                  path_matrix[i][j] = 'l'
            elif (diag >= left and diag >= up):
              if (diag > left and diag > up):
                path_matrix[i][j] = 'd'
              if (path_matrix[i][j] == 'u'):
                if (scoring_matrix[i-1][j-1] > scoring_matrix[i-1][j]):
                  path_matrix[i][j] = 'd'
              if (path_matrix[i][j] == 'l'):
                if (scoring_matrix[i-1][j-1] > scoring_matrix[i][j-1]):
                  path_matrix[i][j] = 'd'
            scoring_matrix[i][j] = max(up,left,diag)
    for i in range(x_size):
      for j in range(y_size):
        if (scoring_matrix[i][j] < 0):
              scoring_matrix[i][j] = 0        

    # TO-DO 4: Find the maximum score and its indexes 
    values = []
    alignment = []
    max_score = scoring_matrix[0][0]
    for i in range(x_size):
      for j in range(y_size):
        if (i > 0 and j > 0):
          if (scoring_matrix[i][j] > max_score):
             max_score = scoring_matrix[i][j]
             max_xcoor = i
             max_ycoor = j
    values.append(max_score)
    algn1 = ""
    algn2 = ""         
    while (max_score > 0):
      if (path_matrix[max_xcoor][max_ycoor] == '?'):
        break
      if (path_matrix[max_xcoor][max_ycoor] == 'u'):
        values.append(scoring_matrix[max_xcoor][max_ycoor-1])
        
        algn1 += "_"
        algn2 += seq[1][max_ycoor-1]
        max_ycoor -= 1
      elif (path_matrix[max_xcoor][max_ycoor] == 'l'):
        values.append(scoring_matrix[max_xcoor-1][max_ycoor])
        algn2 += "_"
        algn1 = seq[0][max_xcoor-1]
        
        max_xcoor -= 1
      elif (path_matrix[max_xcoor][max_ycoor] == 'd'):
        values.append(scoring_matrix[max_xcoor-1][max_ycoor-1])
        
        algn1 += seq[0][max_xcoor-1]
        algn2 += seq[1][max_ycoor-1]
        max_ycoor -= 1
        max_xcoor -= 1  
      max_score = scoring_matrix[max_xcoor][max_ycoor]   
    del values[-1]
    algn1 = algn1[::-1]
    algn2 = algn2[::-1] 
    
    # TO-DO 5: Write your scoring matrix, match score, mismatch score, gap scores, and alignment to an output file
    lf = open("local_aln_output.txt.", "w")
    lf.writelines([algn1, "\n"])
    lf.writelines([algn2, "\n"])
    for line in scoring_matrix:
      lf.writelines([str(line), "\n"])
    lf.writelines(["Gap Extension Score: ", str(gap_extend_penalty), "\n"])
    lf.writelines(["Gap Opening Score: ", str(gap_opening_penalty), "\n"])

    lf.writelines(["Match Score: ", str(match_score), "\n"])
    lf.writelines(["Mismatch Score: ", str(mismatch_score), "\n"])
    lf.close()
    # Use the print_matrix function given above to print your filled matrix
    print_matrix(input_file,scoring_matrix)

#run the global alignment function

# Differences in affine and linear
local_align("test_input1.txt", gap_opening_penalty=-15, gap_extend_penalty=-7, match_score=10, mismatch_score=-2) # affine
local_align("test_input1.txt", gap_opening_penalty=-15, gap_extend_penalty=-15, match_score=10, mismatch_score=-2) # linear

local_align("test_input5.txt", gap_opening_penalty=-15, gap_extend_penalty=-7, match_score=10, mismatch_score=-2)
local_align("test_input5.txt", gap_opening_penalty=-15, gap_extend_penalty=-15, match_score=10, mismatch_score=-2)





          *    A    T    G    A    G    T    C    T    C    T    C    T    G    A    T    A    A    G    G    A    C    A    A    G    G    C    T    G    C    T    G    T    G    A    A    A    G    C    C    C    T    A    T    G    G
    *    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    C    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    T    0    0    8    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    G    0    0    1   18   11    4    0    0    0    0